非标量变量的反向传播，需要先将非标量进行一次sum求和操作，再进行backward反向传播

In [16]:
import torch

In [19]:
x=torch.arange(4.0,requires_grad=True)
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [29]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad,y,y.sum()

(tensor([0., 2., 4., 6.]),
 tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor(14., grad_fn=<SumBackward0>))

分离计算

In [33]:
x.grad.zero_()
y = x * x
u = y.detach()  # 这里的u被detach出去之后就当作常数来看，即u=x²
z = u * x

z.sum().backward()
x.grad,u

(tensor([0., 1., 4., 9.]), tensor([0., 1., 4., 9.]))

In [34]:
x.grad.zero_()
y.sum().backward()  # y是刚刚保留的,y=x²
x.grad

tensor([0., 2., 4., 6.])

Python控制流的梯度计算

In [43]:
def f(a):
    b = a * 2
    while b.norm() < 10:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [44]:
# a = torch.randn(size=(), requires_grad=True)
a = torch.ones(1, requires_grad=True)
d = f(a)
d.backward()
a, d

(tensor([1.], requires_grad=True), tensor([16.], grad_fn=<MulBackward0>))

In [45]:
a.grad, a.grad == d / a

(tensor([16.]), tensor([True]))